In [23]:
import os
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer
from transformers import BertModel
import torch
import torch.nn as nn
from dataset.dataload import load_data,split_data
import random
import json


In [ ]:
class BertNerModel(nn.Module):
    def __init__(self,class_num):
        super().__init__()
        self.bert=BertModel.from_pretrained(os.path.join("bert-base-chinese"))
        self.classifier=nn.Linear(768,class_num)

        self.loss=nn.CrossEntropyLoss()
    def forward(self,batch_index,batch_label=None):
        self.bert(batch_index)
        

In [35]:
class BertDataset(Dataset):
    def __init__(self,texts,lablels,label2index,max_len,tokenizer):
        self.texts=texts
        self.labels=lablels
        self.label2index=label2index
        self.tokenizer=tokenizer
        self.max_len=max_len

    def __getitem__(self, index):
        text=self.texts[index]
        label=self.labels[index][:self.max_len]

        text_index=self.tokenizer.encode(text,add_special_token=True,max_length=self.max_len+2,truncation=True,return_tensors="pt")
        label_index=[0]+[self.label2index[i] for i in label]+[0]*(self.max_len-len(text)+1)
        label_index=torch.tensor(label_index)

        return text_index.reshape(-1),label_index
    def __len__(self):
        return len(self.texts)

In [24]:
texts,labels,label2index,index2label=load_data(os.path.join("data","train_data_public.csv"))
train_text,train_label,vali_text,vali_label=split_data(texts,labels)

In [25]:
with open(os.path.join("data","l2i.txt"),"+a") as f:
    label2index_json = json.dumps(label2index)
    f.write(label2index_json)
with open(os.path.join("data","i2l.txt"),"+a") as f:
    index2label_json = json.dumps(index2label)
    f.write(index2label_json)
with open(os.path.join("data","train_text.txt"),"+a") as f:
    texts_json=json.dumps(train_text,ensure_ascii=False)
    f.write(texts_json)
with open(os.path.join("data","train_label.txt"),"+a") as f:
    labels_json=json.dumps(train_label,ensure_ascii=False)
    f.write(labels_json)
with open(os.path.join("data","vali_text.txt"),"+a") as f:
    texts_json=json.dumps(vali_text,ensure_ascii=False)
    f.write(texts_json)
with open(os.path.join("data","vali_label.txt"),"+a") as f:
    labels_json=json.dumps(vali_label,ensure_ascii=False)
    f.write(labels_json)


In [31]:
def test():
    with open(os.path.join("data","train_text.txt")) as f:
        tmp = f.read()
        train_text=json.loads(tmp)
    with open(os.path.join("data","train_label.txt")) as f:
        tmp = f.read()
        train_label=json.loads(tmp)
    with open(os.path.join("data","vali_text.txt")) as f:
        tmp = f.read()
        vali_text=json.loads(tmp)
    with open(os.path.join("data","vali_label.txt")) as f:
        tmp = f.read()
        vali_label=json.loads(tmp)
    with open(os.path.join("data","i2l.txt")) as f:
        tmp = f.read()
        index2label=json.loads(tmp)
    with open(os.path.join("data","l2i.txt")) as f:
        tmp = f.read()
        label2index=json.loads(tmp)
    return train_text,train_label,vali_text,vali_label,label2index,index2label


In [26]:
with open(os.path.join("data","vali_label.txt"),"+a") as f:
    texts_json=json.dumps(texts,ensure_ascii=False)
    f.write(texts_json)

UnsupportedOperation: not writable

In [33]:
train_text,train_label,vali_text,vali_label,label2index,index2label=test()

In [27]:
with open(os.path.join("data","vali_label.txt"),"r") as f: 
    str = f.read()
    tmp=json.loads(str)

In [37]:
batch_size=10
epoch=100
max_len=100

In [41]:
tokenizer=BertTokenizer.from_pretrained(os.path.join("bert-base-chinese"))
texts,labels,label2index,index2label=load_data()
train_dataset=BertDataset(texts,labels,label2index,max_len,tokenizer)
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=False)